读取数据集、定义神经网络
===========================

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

train_transforms1 = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28, padding=6),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.449, 0.425, 0.392], std=[0.270, 0.267, 0.279])
])
train_transforms2 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(28, padding=6),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.449, 0.425, 0.392], std=[0.270, 0.267, 0.279])
])
training_data1 = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=train_transforms1
)

training_data2 = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

training_data3 = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=train_transforms2
)
training_data = training_data1 + training_data2 + training_data3

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
print(len(train_dataloader))
train_features, train_labels = next(iter(train_dataloader))
print(train_features.size())
test_features, test_labels = next(iter(test_dataloader))
print(test_features.size())

2813
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])


超参数
===============
训练神经网络时的三个基本超参数：
-   **Number of Epochs** - 遍历数据集的次数
-   **Batch Size** - 每次计算梯度时使用的样本数量
-   **Learning Rate** - 学习率.


In [3]:
learning_rate = 0.001
batch_size = 64
epochs = 5

Optimization Loop
=================

每个优化神经网络的循环被称为 **epoch**，每个 epoch 由两部分组成:

-   **训练** - 遍历训练集，优化网络参数。
-   **验证** - 遍历测试集，检查网络性能是否提升。

Pytorch实现了常用的[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)：

[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error)用于回归问题。

[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood)用于分类问题。

[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
是交叉熵损失，结合了 `nn.LogSoftmax` 与 `nn.NLLLoss`。

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

优化器
=========
所有优化计算都被封装在`optimizer` 对象中，
这里使用了随机梯度下降(SGD optimizer)。

Pytorch还内置了多种 [其它优化器](https://pytorch.org/docs/stable/optim.html)
，例如Adam和Rmsprop等。

初始化optimizer对象时，需要传入待优化的模型参数和学习率。

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

在训练循环内，优化过程分为三步:

-调用`optimizer.zero_grad()`重置模型参数的梯度。默认情况下梯度会叠加，为了防止重复计算，需要在每次迭代中显式地将它们清零。

-调用`loss.backward()`来反向传播预测损失。PyTorch记录了损失关于每个参数的梯度。

-调用`optimizer.step()`，使用反向传播中收集的梯度来调整参数。

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 将model设为训练模式，对batch normalization和dropout很重要
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 计算模型输出和损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 将model设为测试模式，对batch normalization和dropout很重要
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # 测试模型时使用 torch.no_grad(), 使测试时不计算梯度
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.313871  [   64/180000]
loss: 2.301575  [ 6464/180000]
loss: 2.295305  [12864/180000]
loss: 2.290795  [19264/180000]
loss: 2.290412  [25664/180000]
loss: 2.279428  [32064/180000]
loss: 2.280367  [38464/180000]
loss: 2.276102  [44864/180000]
loss: 2.284224  [51264/180000]
loss: 2.264475  [57664/180000]
loss: 2.239385  [64064/180000]
loss: 2.225318  [70464/180000]
loss: 2.196508  [76864/180000]
loss: 2.179870  [83264/180000]
loss: 2.194408  [89664/180000]
loss: 2.118066  [96064/180000]
loss: 2.102072  [102464/180000]
loss: 2.109853  [108864/180000]
loss: 2.068735  [115264/180000]
loss: 2.067844  [121664/180000]
loss: 2.079759  [128064/180000]
loss: 2.029583  [134464/180000]
loss: 2.021432  [140864/180000]
loss: 1.974124  [147264/180000]
loss: 1.961333  [153664/180000]
loss: 1.926574  [160064/180000]
loss: 1.956696  [166464/180000]
loss: 1.824554  [172864/180000]
loss: 1.768037  [179264/180000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.78256

课后作业
===============
1. 测试不同的网络结构、优化器、超参数对学习结果的影响；
2. 测试简单的数据增强方案；

课后发送邮件，标题“神经网络实验三 + 姓名 + 学号”, 附件包括代码和实验报告，发送给 xubaile@nju.edu.cn
DDL - 2024.5.12


In [7]:
print(type(train_dataloader))
print(type(training_data))

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataset.ConcatDataset'>
